# Joint Control with UR10 Robot Arm

[![ Click here to deploy.](https://brev-assets.s3.us-west-1.amazonaws.com/nv-lb-dark.svg)](https://brev.nvidia.com/launchable/deploy?launchableID=env-35QaaoiXx6VDmVNBOEtmpLs9VBs)

This notebook demonstrates **joint target control** on a UR10 robot arm loaded from USD. We'll apply sinusoidal trajectories to each joint to create smooth, coordinated motion.

Key topics:
- Loading robots from USD files
- Configuring PD controllers for joint control
- Generating smooth joint trajectories
- Using ArticulationView for batch control


## Setup and Imports


In [ ]:
import newton
import newton.utils
import warp as wp
import numpy as np
from newton.selection import ArticulationView
from tqdm.notebook import trange


## Joint Control Basics

**Joint Target Control** uses PD (Proportional-Derivative) controllers to track desired joint positions:

$$\tau = k_p (q_{target} - q) + k_d (\dot{q}_{target} - \dot{q})$$

Where:
- $q_{target}$: Desired joint position
- $\dot{q}_{target}$: Desired joint velocity
- $q$: Current joint position  
- $k_p$: Position gain (stiffness)
- $k_d$: Velocity gain (damping)

This is the most common control mode for robot arms in manipulation tasks.


## Load UR10 Robot from USD

Download and load the Universal Robots UR10 arm.


In [ ]:
# Download UR10 assets
asset_path = newton.utils.download_asset("universal_robots_ur10")
asset_file = str(asset_path / "usd" / "ur10_instanceable.usda")

# Create model builder
builder = newton.ModelBuilder()

# Register MuJoCo attributes for better dynamics
newton.solvers.SolverMuJoCo.register_custom_attributes(builder)

# Load UR10 from USD
height = 1.2
builder.add_usd(
    asset_file,
    xform=wp.transform(wp.vec3(0.0, 0.0, height)),
    collapse_fixed_joints=False,
    enable_self_collisions=False,
    hide_collision_shapes=True,
)

# Add a pedestal for the robot
builder.add_shape_cylinder(
    -1,  # World body
    xform=wp.transform(wp.vec3(0, 0, height / 2)),
    half_height=height / 2,
    radius=0.08,
)

print(f"UR10 loaded: {builder.body_count} bodies, {builder.joint_count} joints")


## Configure PD Controller Gains

Set stiffness and damping for joint target control.


In [ ]:
# Configure PD gains for all joints
for i in range(len(builder.joint_target_ke)):
    builder.joint_target_ke[i] = 500.0  # Position gain (stiffness)
    builder.joint_target_kd[i] = 50.0   # Velocity gain (damping)

# Set random initial joint configuration
rng = np.random.default_rng(42)
joint_q = rng.uniform(-np.pi, np.pi, builder.joint_dof_count)
builder.joint_q = joint_q.tolist()

# Add ground plane
builder.add_ground_plane()

print(f"PD gains configured: ke=500, kd=50")
print(f"Initial joint config: {builder.joint_dof_count} DOFs")


## Finalize Model and Create Solver


In [ ]:
# Finalize model
model = builder.finalize()

# Create states and control
state_0 = model.state()
state_1 = model.state()
control = model.control()

# Create MuJoCo solver (no contacts needed for this demo)
solver = newton.solvers.SolverMuJoCo(model, disable_contacts=True)

print(f"Model finalized: {model.body_count} bodies")
print(f"Solver: MuJoCo (contacts disabled)")


## Create ArticulationView

Use ArticulationView to access joint properties.


In [ ]:
# Create ArticulationView for the UR10
ur10_view = ArticulationView(
    model,
    "*ur10*",  # Match UR10 articulation
    exclude_joint_types=[newton.JointType.FREE],
)

print(f"ArticulationView created:")
print(f"  Articulation count: {ur10_view.count}")
print(f"  DOFs per articulation: {ur10_view.joint_dof_count}")


## Generate Sinusoidal Trajectories

Create smooth sinusoidal motion for each joint within its limits.


In [ ]:
# Get joint limits and current positions
dof_lower = ur10_view.get_attribute("joint_limit_lower", model)[0].numpy()
dof_upper = ur10_view.get_attribute("joint_limit_upper", model)[0].numpy()
joint_q_current = ur10_view.get_attribute("joint_q", state_0).numpy()[0]

dof_count = ur10_view.joint_dof_count
trajectory_list = []

# Generate sinusoidal trajectory for each DOF
for i in range(dof_count):
    lower = dof_lower[i]
    upper = dof_upper[i]
    
    # Handle unlimited joints (assume angular)
    if not np.isfinite(lower) or abs(lower) > 6.0:
        lower = -np.pi
        upper = np.pi
    
    # Compute phase shift to start at current position
    limit_range = upper - lower
    normalized = (joint_q_current[i] - lower) / limit_range * 2.0 - 1.0
    phase_shift = 0.0
    if abs(normalized) < 1.0:
        phase_shift = np.arcsin(normalized)
    
    # Generate sinusoidal trajectory
    num_steps = int(limit_range * 50)
    t = np.linspace(phase_shift, 2 * np.pi + phase_shift, num_steps)
    traj = np.sin(t) * (upper - lower) * 0.5 + 0.5 * (upper + lower)
    
    trajectory_list.append(traj)

# Stack trajectories: [time_steps, dof_count]
joint_trajectories = np.stack(trajectory_list, axis=1).astype(np.float32)

print(f"Generated trajectories:")
print(f"  Shape: {joint_trajectories.shape} (time_steps × DOFs)")
print(f"  Duration: {len(joint_trajectories)} steps")


## Trajectory Interpolation Kernel

Define a Warp kernel to interpolate trajectory waypoints.


In [ ]:
@wp.kernel
def update_joint_targets_kernel(
    trajectory: wp.array2d(dtype=wp.float32),  # [time_steps, dof_count]
    time_idx: wp.array(dtype=wp.float32),      # [1] - current time index
    dt: float,                                  # Time step
    speed: float,                               # Playback speed
    # output
    joint_targets: wp.array(dtype=wp.float32), # [dof_count]
):
    # Interpolate joint targets from trajectory.
    dof = wp.tid()
    
    # Update time index
    t = time_idx[0]
    t = wp.mod(t + dt * speed, float(trajectory.shape[0] - 1))
    if dof == 0:  # Only update once
        time_idx[0] = t
    
    # Linear interpolation between waypoints
    step = int(t)
    frac = wp.frac(t)
    
    val0 = trajectory[step, dof]
    val1 = trajectory[step + 1, dof]
    
    joint_targets[dof] = wp.lerp(val0, val1, frac)


# Convert trajectory to Warp array
trajectory_wp = wp.array(joint_trajectories, dtype=wp.float32)
time_idx = wp.zeros(1, dtype=wp.float32)

print("Trajectory interpolation kernel defined")


## Simulation with CUDA Graph

Capture the simulation loop for optimal performance.


In [ ]:
# Simulation parameters
fps = 50
frame_dt = 1.0 / fps
sim_substeps = 10
sim_dt = frame_dt / sim_substeps
control_speed = 50.0  # Trajectory playback speed

# Get joint target array from control
joint_targets = control.joint_target_pos

def simulate_step():
    # Run physics substeps with joint control.
    global state_0, state_1
    for _ in range(sim_substeps):
        state_0.clear_forces()
        
        # Update joint targets from trajectory
        wp.launch(
            update_joint_targets_kernel,
            dim=dof_count,
            inputs=[trajectory_wp, time_idx, sim_dt, control_speed],
            outputs=[joint_targets],
        )
        
        # Step solver
        solver.step(state_0, state_1, control, None, sim_dt)
        
        # Swap states
        state_0, state_1 = state_1, state_0

# Capture CUDA graph
graph = None
if wp.get_device().is_cuda and wp.is_mempool_enabled(wp.get_device()):
    print("Capturing CUDA graph...")
    with wp.ScopedCapture() as capture:
        simulate_step()
    graph = capture.graph
    print("CUDA graph captured successfully")
else:
    print("Running on CPU (no CUDA graph)")

print(f"\nSimulation: {fps} Hz, Substeps: {sim_substeps}")


## Run Simulation

Execute the simulation and visualize the robot motion.


In [ ]:
# Create viewer
viewer = newton.viewer.ViewerRerun(keep_historical_data=True)
viewer.set_model(model)

# Main simulation loop
num_frames = 300
sim_time = 0.0

for frame in trange(num_frames, desc="Simulating UR10"):
    # Run physics (use CUDA graph if available)
    if graph:
        wp.capture_launch(graph)
    else:
        simulate_step()
    
    # Log to viewer
    viewer.begin_frame(sim_time)
    viewer.log_state(state_0)
    viewer.end_frame()
    
    sim_time += frame_dt

print(f"\nSimulation complete! Total time: {sim_time:.2f} seconds")
viewer


## Inspect Joint Tracking

Let's examine how well the robot tracks the target trajectories.


In [ ]:
# Get current joint positions and targets
joint_pos = state_0.joint_q.numpy()
joint_targets_np = joint_targets.numpy()

print("Current Joint State:")
for i in range(dof_count):
    print(f"  Joint {i}: pos={joint_pos[i]:.3f}, target={joint_targets_np[i]:.3f}, "
          f"error={abs(joint_pos[i] - joint_targets_np[i]):.4f}")

# Compute tracking error
tracking_error = np.abs(joint_pos[:dof_count] - joint_targets_np)
print(f"\nTracking Performance:")
print(f"  Mean error: {np.mean(tracking_error):.4f} rad ({np.degrees(np.mean(tracking_error)):.2f}°)")
print(f"  Max error: {np.max(tracking_error):.4f} rad ({np.degrees(np.max(tracking_error)):.2f}°)")


## Summary

In this notebook, we demonstrated:

1. **Loading robots from USD** with `add_usd()`
2. **Configuring PD controllers** with stiffness and damping gains
3. **Generating smooth trajectories** using sinusoidal functions
4. **Joint target control** via `control.joint_target_pos`
5. **CUDA graph optimization** for fast simulation

### Key Concepts

**PD Controller:**
```python
builder.joint_target_ke[i] = 500.0  # Stiffness
builder.joint_target_kd[i] = 50.0   # Damping
```

Higher gains → stiffer tracking, but can cause instability.

**Joint Control Modes:**
- **Position control**: Track target positions (this notebook)
- **Velocity control**: Track target velocities
- **Torque control**: Direct torque commands

**Trajectory Generation:**
- Sinusoidal: Smooth periodic motion
- Spline: Smooth interpolation between waypoints
- Minimum jerk: Optimal smoothness

**ArticulationView Benefits:**
- Batch access to joint properties
- Efficient for multi-world scenarios
- Clean API for robot manipulation

### Applications

- **Pick and place**: Move to grasp poses
- **Teleoperation**: Follow human commands
- **Trajectory optimization**: Execute planned paths
- **Imitation learning**: Replay demonstrations

### Next Steps

- Add end-effector targets with IK
- Implement velocity or torque control
- Create more complex trajectories (circles, writing)
- Add collision avoidance constraints
